In [1]:
# Define the SturzHeuristik class with modular, multivariate heuristic features

import numpy as np
from scipy.spatial import ConvexHull

class SturzHeuristik:
    def __init__(self, kpts, frame_shape):
        self.kpts = np.array([pt for pt in kpts if pt[0] > 0 and pt[1] > 0])
        self.frame_shape = frame_shape
        self.features = {}

    def compute_all(self):
        if len(self.kpts) < 10:
            self.features["valid"] = False
            return self.features

        self.features["valid"] = True
        self.features.update({
            "dispersion": self._dispersion(),
            "hull_area": self._hull_area(),
            "centroid_y": self._centroid_y(),
            "symmetry": self._symmetry(),
            "extreme_spread": self._extreme_spread(),
            "horizontalness": self._horizontalness(),
            "fragmentation": self._fragmentation(),
            "compactness": self._compactness(),
            "limb_density": self._limb_density(),
            "height_spread": self._height_spread(),
            "feet_distance": self._feet_distance(),
            "hand_below_hip": self._hand_support(),
            "torso_kink": self._torso_bend(),
        })
        self.features["SMI"] = self._combine_score()
        return self.features

    def _dispersion(self):
        dists = [np.linalg.norm(k1 - k2) for i, k1 in enumerate(self.kpts) for k2 in self.kpts[i+1:]]
        return np.mean(dists)

    def _hull_area(self):
        if len(self.kpts) >= 3:
            hull = ConvexHull(self.kpts)
            return hull.area
        return 1e6

    def _centroid_y(self):
        return np.mean(self.kpts[:, 1]) / self.frame_shape[0]

    def _symmetry(self):
        if len(self.kpts) > 12:
            left = self.kpts[5:9]
            right = self.kpts[6:10]
            return np.mean([np.abs(np.linalg.norm(l - r)) for l, r in zip(left, right)])
        return 0

    def _extreme_spread(self):
        min_pt = np.min(self.kpts, axis=0)
        max_pt = np.max(self.kpts, axis=0)
        return np.linalg.norm(max_pt - min_pt)

    def _horizontalness(self):
        kpts_centered = self.kpts - np.mean(self.kpts, axis=0)
        u, s, vh = np.linalg.svd(kpts_centered)
        direction = vh[0]
        return np.abs(direction[0])

    def _fragmentation(self):
        k0 = self.kpts[0]
        return np.mean([np.linalg.norm(k - k0) for k in self.kpts[1:]])

    def _compactness(self):
        try:
            area = self._hull_area()
            bbox = np.ptp(self.kpts, axis=0)
            rect_area = bbox[0] * bbox[1] + 1e-6
            return area / rect_area
        except:
            return 0

    def _limb_density(self):
        center = np.mean(self.kpts, axis=0)
        dists = np.linalg.norm(self.kpts - center, axis=1)
        return np.std(dists)

    def _height_spread(self):
        ys = self.kpts[:, 1]
        return np.ptp(ys)

    def _feet_distance(self):
        if len(self.kpts) > 15:
            return np.linalg.norm(self.kpts[15] - self.kpts[16])
        return 0

    def _hand_support(self):
        if len(self.kpts) > 12:
            hip_y = np.mean([self.kpts[11][1], self.kpts[12][1]])
            return float(self.kpts[9][1] > hip_y and self.kpts[10][1] > hip_y)
        return 0

    def _torso_bend(self):
        if len(self.kpts) > 12:
            shoulder_center = (self.kpts[5] + self.kpts[6]) / 2
            hip_center = (self.kpts[11] + self.kpts[12]) / 2
            return np.linalg.norm(shoulder_center - hip_center)
        return 0

    def _combine_score(self):
        # Normalize & combine selected features
        norm = lambda x, low, high: min(max((x - low) / (high - low + 1e-6), 0), 1)
        smi = (
            0.2 * norm(self.features["centroid_y"], 0.6, 1.0) +
            0.15 * norm(self.features["horizontalness"], 0.7, 1.0) +
            0.15 * norm(self.features["compactness"], 0.0, 0.3) +
            0.15 * norm(self.features["limb_density"], 0.0, 50) +
            0.1 * norm(self.features["hand_below_hip"], 0.0, 1.0) +
            0.1 * norm(self.features["feet_distance"], 100, 200) +
            0.15 * norm(self.features["fragmentation"], 0, 200)
        )
        return smi


import pandas as pd
features_df = pd.DataFrame(columns=["Feature", "Wert"])
features_df


import cv2
from ultralytics import YOLO
import numpy as np

# Die Klasse SturzHeuristik wird als vorhanden vorausgesetzt (bereits im Usercode integriert)

# Modell laden
model = YOLO("yolo11n-pose.pt")
cap = cv2.VideoCapture("5_Video/chairlift_vid1.mp4")

POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (0, 5), (5, 6), (6, 7), (7, 8),
    (5, 11), (6, 12), (11, 12),
    (11, 13), (13, 15), (12, 14), (14, 16)
]

# Ergebnis-Logging zur Visualisierung
frame_smi_data = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)

    for r in results:
        kpts_tensor = r.keypoints.xy
        for person_kpts in kpts_tensor:
            person_kpts = person_kpts.cpu().numpy()
            if person_kpts.shape[0] < 17:
                continue

            heuristik = SturzHeuristik(person_kpts, frame.shape)
            features = heuristik.compute_all()

            if not features["valid"]:
                continue

            smi = features["SMI"]
            is_fall = smi > 0.7
            frame_smi_data.append(smi)

            point_color = (0, 255, 0) if not is_fall else (0, 0, 255)
            line_color = (255, 0, 0) if not is_fall else (0, 0, 255)

            for x, y in person_kpts:
                if x > 0 and y > 0:
                    cv2.circle(frame, (int(x), int(y)), 3, point_color, -1)

            for pair in POSE_CONNECTIONS:
                x1, y1 = person_kpts[pair[0]]
                x2, y2 = person_kpts[pair[1]]
                if x1 > 0 and y1 > 0 and x2 > 0 and y2 > 0:
                    cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), line_color, 2)

            if is_fall:
                x, y = int(person_kpts[0][0]), int(person_kpts[0][1])
                cv2.putText(frame, f"STURZ erkannt (SMI {smi:.2f})", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow("YOLO Pose mit SturzHeuristik", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


IndexError: index 16 is out of bounds for axis 0 with size 16